In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Connection parameters
host = 'evchargerdb.cjlsbfswobs8.us-east-1.rds.amazonaws.com'
database = 'evchargerinst'
user = 'postgres'
password = 'charger2024'
port = '5432'

# Establishing a connection
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connection to PostgreSQL database successful")
except psycopg2.Error as e:
    print("Error: Could not connect to PostgreSQL database")
    print(e)

Connection to PostgreSQL database successful


In [3]:
query = 'SELECT * FROM "evChargers"."EV_Charger_Extract";'

In [4]:
df = pd.read_sql_query(query, conn)

C:\Users\alawr\AppData\Local\Temp\ipykernel_18564\1553155693.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [5]:
df.head()

,Unnamed: 0,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,...,nps_unit_name,access_days_time_fr,intersection_directions_fr,bd_blends_fr,groups_with_access_code_fr,ev_pricing_fr,ev_network_ids,distance,distance_km,federal_agency
0,0,public,24 hours daily,None,None,2024-04-11,None,ELEC,Public,194170,...,None,None,None,None,Public,None,{'station': ['4ed29a8a-6b2d-4707-b63d-c6b70d26...,4.92687,7.92903,None
1,1,public,None,None,None,2022-10-06,None,ELEC,Public,114970,...,None,None,None,None,Public,None,{'station': ['503452']},3.48093,5.60201,None
2,2,public,None,None,None,2022-10-06,None,ELEC,Public,115166,...,None,None,None,None,Public,None,{'station': ['511149']},0.81104,1.30524,None
3,3,public,None,None,None,2022-10-06,None,ELEC,Public,122473,...,None,None,None,None,Public,None,{'station': ['506790']},2.34197,3.76904,None
4,4,public,24 hours daily,None,None,2021-09-10,None,ELEC,Public,194700,...,None,None,None,None,Public,None,None,3.07841,4.95422,None


In [27]:
select_columns = df[['station_name',
 'street_address',
 'city',
 'state',
 'zip',
 'country',
 'facility_type',
 'latitude',
 'longitude',
 'ev_network']]

select_columns.head()

,station_name,street_address,city,state,zip,country,facility_type,latitude,longitude,ev_network
0,101 E 78th St,101 E 78th St,New York,NY,10075,US,None,40.774878,-73.960671,FLO
1,Little Man Parking (Queens West) - Tesla Desti...,4-76 47th Ave,Long Island City,NY,11109,US,PAY_GARAGE,40.745627,-73.955638,Tesla Destination
2,iPark 59-63 Allen Street - Tesla Destination,59-63 Allen St,New York,NY,10002,US,PAY_GARAGE,40.717143,-73.991597,Tesla Destination
3,iPark 34 N 7th Street 2 - Tesla Destination,34 N 7th St,Brooklyn,NY,11249,US,None,40.720708,-73.962310,Tesla Destination
4,Revel Brooklyn Superhub,467 Marcy Ave,Brooklyn,NY,11206,US,PARKING_LOT,40.699334,-73.949665,EVGATEWAY


In [10]:
select_columns.loc[select_columns['state'] == 'NJ', 'zip'] = select_columns.loc[select_columns['state'] == 'NJ', 'zip'].apply(lambda x: str(x).zfill(5))

C:\Users\alawr\AppData\Local\Temp\ipykernel_18564\3786376129.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['07030' '07030' '07030' '07030' '07093' '07087' '07030' '07030' '07030'
 '07093' '07093' '07030' '07030' '07201' '07047' '07030' '07093' '07030'
 '07307' '07030' '07030' '07030' '07030' '07030' '07030' '07030' '07030'
 '07201' '07030' '07030' '07093' '07093' '07093' '07093' '07093' '07030'
 '07030' '07030' '07086' '07047' '07047' '07087' '07087' '07307' '07030'
 '07310' '07310' '07310' '07307' '07310' '07310' '07032' '07310' '07310'
 '07310' '07302' '07311' '07302' '07302' '07302' '07302' '07302' '07302'
 '07302' '07302' '07302' '07302' '07302' '07302' '07302' '07302' '07311'
 '07032' '07311' '07302' '07302' '07302' '07302' '07302' '07302' '07306'
 '07306' '07302' '07392' '07304' '07305' '07306' '07306' '07304' '07304'
 '07305' '07305' '07305' '07305' '07305' '07305' '07304' '07304' '07032'
 '07305' '07

In [17]:
filtered_rows = select_columns[select_columns['state'] == 'NJ']
filtered_rows.head()

,station_name,street_address,city,state,zip,station_name,country,facility_type,latitude,longitude,ev_network
45,W Hoboken - Tesla Destination,225 River St,Hoboken,NJ,07030,W Hoboken - Tesla Destination,US,HOTEL,40.739190,-74.027760,Tesla Destination
101,DT&P GARAGE B ST2,28 2nd St,Hoboken,NJ,07030,DT&P GARAGE B ST2,US,None,40.738262,-74.029045,ChargePoint Network
106,Littleman Parking 2 12th Street - Tesla Destin...,2 12th St,Hoboken,NJ,07030,Littleman Parking 2 12th Street - Tesla Destin...,US,None,40.750728,-74.024441,Tesla Destination
107,THE BERKSHIRE STATION 1,1401 Hudson St,Hoboken,NJ,07030,THE BERKSHIRE STATION 1,US,None,40.753210,-74.024610,ChargePoint Network
108,RIVERBEND 1 RIVERBEND,25 Ave at Port Imperial,West New York,NJ,07093,RIVERBEND 1 RIVERBEND,US,None,40.779675,-74.010070,ChargePoint Network


In [28]:
ev_charging_updated_df = select_columns
ev_charging_updated_df.head()

,station_name,street_address,city,state,zip,country,facility_type,latitude,longitude,ev_network
0,101 E 78th St,101 E 78th St,New York,NY,10075,US,None,40.774878,-73.960671,FLO
1,Little Man Parking (Queens West) - Tesla Desti...,4-76 47th Ave,Long Island City,NY,11109,US,PAY_GARAGE,40.745627,-73.955638,Tesla Destination
2,iPark 59-63 Allen Street - Tesla Destination,59-63 Allen St,New York,NY,10002,US,PAY_GARAGE,40.717143,-73.991597,Tesla Destination
3,iPark 34 N 7th Street 2 - Tesla Destination,34 N 7th St,Brooklyn,NY,11249,US,None,40.720708,-73.962310,Tesla Destination
4,Revel Brooklyn Superhub,467 Marcy Ave,Brooklyn,NY,11206,US,PARKING_LOT,40.699334,-73.949665,EVGATEWAY


In [23]:
ev_charging_updated_df.rename(columns={'station_name': 'station_names'}, inplace=True)

C:\Users\alawr\AppData\Local\Temp\ipykernel_18564\744879987.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_charging_updated_df.rename(columns={'station_name': 'station_names'}, inplace=True)


In [29]:
ev_charging_updated_df.head()

,station_name,street_address,city,state,zip,country,facility_type,latitude,longitude,ev_network
0,101 E 78th St,101 E 78th St,New York,NY,10075,US,None,40.774878,-73.960671,FLO
1,Little Man Parking (Queens West) - Tesla Desti...,4-76 47th Ave,Long Island City,NY,11109,US,PAY_GARAGE,40.745627,-73.955638,Tesla Destination
2,iPark 59-63 Allen Street - Tesla Destination,59-63 Allen St,New York,NY,10002,US,PAY_GARAGE,40.717143,-73.991597,Tesla Destination
3,iPark 34 N 7th Street 2 - Tesla Destination,34 N 7th St,Brooklyn,NY,11249,US,None,40.720708,-73.962310,Tesla Destination
4,Revel Brooklyn Superhub,467 Marcy Ave,Brooklyn,NY,11206,US,PARKING_LOT,40.699334,-73.949665,EVGATEWAY


In [30]:

# Construct the connection string
db_connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Define the schema name and view name
schema_name = 'evChargers'
table_name = 'EV_Charging_Final'

# Write the DataFrame as a view to the database
ev_charging_updated_df.to_sql(table_name, con=create_engine(db_connection_str), schema=schema_name, if_exists='replace', index=False)


61